In [1]:
from datetime import datetime
import requests
from enum import Enum
from pydantic import BaseModel, Field
from typing import Any, List, Optional

In [2]:
file_uri = '/path/to/file'
mask1 = {'fill': 'toself', 'x': [0, 1, 2, 0], 'y': [0, 2, 0, 0], 'type': 'scatter'}
mask2 = {'fill': 'toself', 'x': [3,3,5,5,3], 'y': [0.5,1.5,1.5,0.5,0.5], 'type': 'scatter'}
SPLASH_URI = 'http://localhost:8087/api/v0'

In [3]:
class Locator(BaseModel):
    spec: str = Field(description="Description of the specification for this locator")
    path: Any = Field(description="Locator information defined by the spec field")


class TaggingEvent(BaseModel, extra='forbid'):
    tagger_id: str
    run_time: datetime
    accuracy: Optional[float] = Field(ge=0.0, le=1.0, default=None)


class Tag(BaseModel):
    name: str = Field(description="name of the tag")
    locator: Optional[Locator] = Field(description="optional location information, "
                                                   "for indicating a subset of a dataset that "
                                                   "this tag applies to", default=None)

    confidence: Optional[float] = Field(description="confidence provided for this tag", default=None)
    event_id: Optional[str] = Field(description="id of event where this tag was created", default=None)


class DatasetType(str, Enum):
    tiled = "tiled"
    file = "file"
    web = "web"


class Dataset(BaseModel):
    project: str = None
    type: DatasetType = None
    uri: str = None
    tags: Optional[List[Tag]] = None

### Posting a new tagging event for version control

In [4]:
new_tagging_event = TaggingEvent(tagger_id='username', 
                                 run_time=datetime.utcnow())
response = requests.post(f'{SPLASH_URI}/events', json=new_tagging_event.model_dump(mode='json'))
print(response.json())
event_id = response.json()['uid']


{'uid': '2d9f1c33-8056-40fa-8f60-44672c9238bb'}


### Posting a data set with a set of tags

In [5]:
dataset = Dataset(
    uri=file_uri,
    type='file',
    project='project_id',    # Project ID groups a set of datasets
    tags=[
        Tag(
            name='class_name_1',
            locator=Locator(
                spec='plotly_annotation', 
                path=mask1
                ),
            event_id=event_id
        ),
        Tag(
            name='class_name_2',
            locator=Locator(
                spec='plotly_annotation', 
                path=mask2
                ),
            event_id=event_id
        )
    ]
    )

response = requests.post(f'{SPLASH_URI}/datasets', json=[dataset.model_dump(mode='json')])
status = response.status_code
dataset_uid = response.json()[0]['uid']
print(f'Status code: {status}, dataset uid: {dataset_uid}')

Status code: 200, dataset uid: 38d1fff3-a480-4ee8-bb65-8c61d1381bcd


### Quering datasets

In [6]:
response = requests.get(f'{SPLASH_URI}/datasets', 
                        params={"page[offset]": 0,      # offset for query
                                "page[limit]": 10,      # length of queried items
                                "uri": file_uri,
                                "event_id": event_id})
splash_dataset = response.json()

print(f'splash_dataset retrieved with query: {splash_dataset}')

splash_dataset retrieved with query: [{'uid': '38d1fff3-a480-4ee8-bb65-8c61d1381bcd', 'schema_version': '1.3', 'project': 'project_id', 'type': 'file', 'uri': '/path/to/file', 'tags': [{'uid': '7e350b1e-85c5-4d2d-93f1-0f65419a8bf8', 'name': 'class_name_1', 'locator': {'spec': 'plotly_annotation', 'path': {'fill': 'toself', 'x': [0, 1, 2, 0], 'y': [0, 2, 0, 0], 'type': 'scatter'}}, 'confidence': None, 'event_id': '2d9f1c33-8056-40fa-8f60-44672c9238bb'}, {'uid': '6ce41e7e-093d-44a8-9c38-87b1f9533572', 'name': 'class_name_2', 'locator': {'spec': 'plotly_annotation', 'path': {'fill': 'toself', 'x': [3, 3, 5, 5, 3], 'y': [0.5, 1.5, 1.5, 0.5, 0.5], 'type': 'scatter'}}, 'confidence': None, 'event_id': '2d9f1c33-8056-40fa-8f60-44672c9238bb'}]}]


### New tagging event with new version of tags

In [7]:
mask3 = {'fill': 'toself', 'x': [3,3,5,1,3], 'y': [0.5,1,1.5,0.5,0.5], 'type': 'scatter'}

new_tagging_event = TaggingEvent(
    tagger_id='username',
    run_time=datetime.utcnow()
)
response = requests.post(f'{SPLASH_URI}/events', json=new_tagging_event.model_dump(mode='json'))
event_id2 = response.json()['uid']

### Add new tag to existing dataset

In [8]:
new_tag = Tag(
            name='class_name_3',
            event_id=event_id2,
            locator=Locator(
                spec='plotly_annotation',
                path=mask3)
            )

response = requests.patch(f'{SPLASH_URI}/datasets/{dataset_uid}/tags', 
                          json={'add_tags': [new_tag.model_dump(mode='json')]})
status = response.status_code
tags_uids = response.json()
print(f'Status code: {status}, the uids of the added tags are: {tags_uids["added_tags_uid"]}')

Status code: 200, the uids of the added tags are: ['06aa5fd1-36a6-4706-b549-eb9cf3be5ca1']
